## **P6 (Наука, освіта, охорона здоров’я, культура, фізкультура і спорт, материнство і дитинство, сім’я та молодь): 9/9**

In [15]:
import pandas as pd
from custom_func import *

### **p6_01**
#### частка укладених декларацій про вибір лікаря, який надає первинну медичну допомогу відносно загальної кількості населення в області

In [16]:
df_P06_007 = pd.read_csv(f"{INPUTS_PATH}/P6/P06_007.csv", dtype={"legal_entity_edrpou": str})
df_P06_007["legal_entity_edrpou"] = df_P06_007["legal_entity_edrpou"].str.zfill(8)
df_P06_007 = df_P06_007.loc[df_P06_007["division_area"].ne("М.КИЇВ")].copy()

In [17]:
df_P06_007_fin = df_P06_007.groupby("division_area", as_index=False)["count_declarations"].sum() \
                           .rename(columns={"division_area": "region"})
df_P06_007_fin["region"] = df_P06_007_fin["region"].str.title()
df_P06_007_fin["population"] = df_P06_007_fin["region"].map(dict_population)
df_P06_007_fin['p6_01_raw'] = df_P06_007_fin["count_declarations"]/df_P06_007_fin["population"]

normalize_parameter(df_P06_007_fin, "p6_01_raw", "p6_01",min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_01 нормалізовано і додано до таблиці



### **p6_02**
#### рівень охоплення щепленнями відповідно до річного календаря профілактичних щеплень в Україні
### **p6_03**
#### частка лікувально-профілактичних закладів, що мають статус некомерційних комунальних підприємств
### **p6_04**
#### частка лікувально-профілактичних закладів, підключених до системи e-health
### **p6_09**
#### частка кількість дітей віком від трьох до п’яти років, охоплених дошкільною освітою

In [18]:
df_KMU["p6_02_raw"] = df_KMU["Щеплення_факт"]/df_KMU["Щеплення_передбачено"]
df_KMU["p6_03_raw"] = df_KMU["КНП"]/df_KMU["Лікарень (автономізація)"]
df_KMU["p6_04_raw"] = df_KMU["EHEALTH"]/df_KMU["Лікарень (комп'ютеризація)"]
df_KMU["p6_09_raw"] = df_KMU["Охоплення дошк освітою"] / df_KMU["Кількість дітей"]

normalize_parameter(df_KMU, "p6_02_raw", "p6_02",min_bound=0,max_bound=1)
normalize_parameter(df_KMU, "p6_03_raw", "p6_03",min_bound=0,max_bound=1)
normalize_parameter(df_KMU, "p6_04_raw", "p6_04",min_bound=0,max_bound=1)
normalize_parameter(df_KMU, "p6_09_raw", "p6_09",min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_02 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_03 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_04 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_09 нормалізовано і додано до таблиці



### **p6_05**
#### середній розмір класу у сільських закладах загальної середньої освіти (ЗЗСО)
### **p6_06**
#### частка ЗЗСО, у яких навчається менше 100 учнів, за винятком ЗЗСО І та ІІІ ступеня
### **p6_07**
#### кількість учнів, що навчаються індивідуально не за станом здоров’я

In [19]:
df_P06_005 = pd.read_csv(f"{INPUTS_PATH}/P6/P06_005.csv")

#p6_05 середній розмір класу у сільських закладах загальної середньої освіти (ЗЗСО)
df_P06_005_classes = df_P06_005.loc[df_P06_005['Тип місцевості']=='сільска',['Область','ЗНЗ1 1.1.1','ЗНЗ1 1.5.1']]
df_P06_005_classes['size'] = (df_P06_005_classes['ЗНЗ1 1.5.1']/df_P06_005_classes['ЗНЗ1 1.1.1']).map(lambda x: np.nan if x==np.inf else x)
df_P06_005_classes = df_P06_005_classes.groupby('Область')['size'].mean().rename('p6_05_raw')

#p6_06 частка ЗЗСО, у яких навчається менше 100 учнів, за винятком ЗЗСО І та ІІІ ступеня
df_P06_005_small = df_P06_005.loc[(~df_P06_005['Ступiнь школи'].isin(['I', np.nan, 'III']))&(df_P06_005['Форма власностi']!='приватна'),['Область','ЗНЗ1 1.5.1','ЗНЗ1 1.16.1']]
df_P06_005_small['all'] = df_P06_005_small['ЗНЗ1 1.5.1']+df_P06_005_small['ЗНЗ1 1.16.1']
df_P06_005_small = pd.concat([df_P06_005_small.loc[df_P06_005_small['all']<=100].groupby('Область').size().rename('small'),df_P06_005_small.groupby('Область').size().rename('all')],axis=1)
df_P06_005_small['p6_06_raw'] = df_P06_005_small['small']/df_P06_005_small['all']

#p6_07 кількість учнів, що навчаються індивідуально не за станом здоров’я
df_P06_005_ind_study = df_P06_005[['Область','ЗНЗ1 1.5.1','ЗНЗ1 1.10.1','ЗНЗ1 1.13.1']]
df_P06_005_ind_study = df_P06_005_ind_study.groupby('Область').sum()
df_P06_005_ind_study['p6_07_raw'] = (df_P06_005_ind_study['ЗНЗ1 1.10.1']-df_P06_005_ind_study['ЗНЗ1 1.13.1'])/df_P06_005_ind_study['ЗНЗ1 1.5.1']

df_P06_005_fin = pd.concat([df_P06_005_classes,df_P06_005_small,df_P06_005_ind_study],axis=1,sort=True)
df_P06_005_fin = df_P06_005_fin.drop('м.Київ').reset_index().rename(columns={'index':'region'})
df_P06_005_fin['region'] = df_P06_005_fin['region'].map(lambda x: x.split(' ')[0])

normalize_parameter(df_P06_005_fin,'p6_05_raw','p6_05',min_bound=0,max_bound=20)
normalize_parameter(df_P06_005_fin,'p6_06_raw','p6_06',min_bound=1,max_bound=0)
normalize_parameter(df_P06_005_fin,'p6_07_raw','p6_07')

/mnt/sdb1/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Емпіричні границі: (0, 20)
Коефіцієнти нормалізації: (0.05, -0.0)
Параметр p6_05 нормалізовано і додано до таблиці

Емпіричні границі: (1, 0)
Коефіцієнти нормалізації: (-1.0, 1.0)
Параметр p6_06 нормалізовано і додано до таблиці

Емпіричні границі: (0.0037858811120319445, 0.024049689941209634)
Коефіцієнти нормалізації: (49.34906405947279, -0.1868296895192125)
Параметр p6_07 нормалізовано і додано до таблиці



### **p6_08**
#### частка ЗЗСО ІІ та/або ІІІ ступеня, у яких бракує профільних вчителів з двох чи більше предметів (української мови та літератури, історії, фізики, математики, основ інформатики, хімії, географії, біології)

In [20]:
'''
всі 2.5.1
українська мова та література 2.5.2
історія 2.5.5
фізика 2.5.6
математика 2.5.7
основи інформатики 2.5.8
хімія 2.5.9
географія 2.5.10
біологія 2.5.11
'''
df_P06_006 = pd.read_csv(f"{INPUTS_PATH}/P6/P06_006.csv",sep='{',low_memory=False)

df_P06_006.columns[:50]
cols_gen = ['Назва','Ступiнь школи','Форма власностi','Активна(1)не активна(0)','Область','Район','Населенный пункт']
cols_rvk = ['83-РВК 2.5.1','83-РВК 2.5.2','83-РВК 2.5.5','83-РВК 2.5.6','83-РВК 2.5.7','83-РВК 2.5.8','83-РВК 2.5.9','83-РВК 2.5.10','83-РВК 2.5.11']

df_P06_006_sel = df_P06_006[cols_gen+cols_rvk]
print(df_P06_006_sel.shape)
df_P06_006_sel = df_P06_006_sel.loc[(df_P06_006['83-РВК 2.5.1']!=0)&(df_P06_006_sel['Форма власностi']!='приватна')&(df_P06_006_sel['Активна(1)не активна(0)']==1)&(df_P06_006_sel['Ступiнь школи'].isin(['I-III','III', 'II-III']))]
df_P06_006_sel = df_P06_006_sel.loc[~df_P06_006_sel['Назва'].str.contains('МНВК|комб|міжшк')]
print(df_P06_006_sel.shape)
df_P06_006_sel['no_teachers_subj'] = df_P06_006_sel.isin([0]).sum(axis=1)

#за регіонами: відбір шкіл, де бракує 2 і більше профільних вчителів та усіх шкіл
schools_no_teachers = df_P06_006_sel.loc[df_P06_006_sel['no_teachers_subj']>1].groupby('Область').size().rename('schools_no_teachers')
schools_all = df_P06_006_sel.groupby('Область').size().rename('schools_all')

df_P06_006_fin = pd.concat([schools_no_teachers,schools_all],axis=1)
df_P06_006_fin['p6_08_raw'] = df_P06_006_fin['schools_no_teachers']/df_P06_006_fin['schools_all']
df_P06_006_fin = df_P06_006_fin.drop('м.Київ').reset_index().rename(columns={'Область':'region'})
df_P06_006_fin['region'] = df_P06_006_fin['region'].map(lambda x: x.split(' ')[0])
normalize_parameter(df_P06_006_fin,'p6_08_raw','p6_08',min_bound=0,max_bound=1)

(15216, 16)
(9916, 16)
Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p6_08 нормалізовано і додано до таблиці



### **Група 1:** p6_01-p6_04, охорона здоров'я
### **Група 2:** p6_05-p6_09, освіта

In [21]:
dict_weights = {
    'p6_01':1,
    'p6_02':1,
    'p6_03':1,
    'p6_04':1,
    'p6_05':1,
    'p6_06':1,
    'p6_07':1,
    'p6_08':1,
    'p6_09':1
}

sources = [df_P06_007_fin, df_KMU, df_P06_005_fin, df_P06_006_fin]
save_data(sources,regions,dict_weights,'P6')

Дані за галузевим параметром P6 збережено
               region     p6_01     p6_02     p6_03     p6_04     p6_09  \
0           Волинська  0.838103  0.620203  1.000000  0.978723  0.801450   
1           Вінницька  0.855345  0.662857  1.000000  1.000000  0.871302   
2    Дніпропетровська  0.786499  0.760032  0.978571  0.978571  0.912871   
3            Донецька  0.331355  0.883241  1.000000  1.000000  0.967810   
4         Житомирська  0.821836  0.278326  1.000000  1.000000  0.955767   
5        Закарпатська  0.797767  0.244748  1.000000  1.000000  0.771542   
6          Запорізька  0.785221  0.597770  0.983333  0.983333  0.849688   
7            Київська  0.856180  0.309781  1.000000  1.000000  0.960629   
8   Івано-Франківська  0.815657  0.132746  1.000000  1.000000  0.727526   
9      Кіровоградська  0.751610  0.283158  1.000000  1.000000  0.929813   
10          Луганська  0.225637  0.030883  1.000000  1.000000  0.795118   
11          Львівська  0.841911  0.270156  0.979592  1.000